In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [2]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Text chunking configuration


In [3]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("aws_documentation_test")

In [4]:
#for ingestion pipeline we can set up a s3 bucket. user inputs their files -> sent to s3 bucket. We retrieve file from s3 and put it through the pipeline. 
documents = (
    SimpleDirectoryReader(input_dir="C:/Users/Adel/Desktop/aws-documentation-main/aws-documentation-main/documents/alexa-for-business-administration-guide/doc_source", recursive=False, required_exts=[".md" , ".pdf"]).load_data())


In [5]:
from llama_index.llms import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf" #using 13b for the time being until i can confirm that everything is working 100%
##you can change to 70b by just replacing the linkg

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '5120', 'llama.block_count': '40', 'llama.feed_forward_length': '13824', 'llama.attention.head_count': '40', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '2', 'llama.attention.head_count_kv': '40', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}


In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(
    chunk_size=500,
    llm=llm,
    embed_model=embed_model
)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)
#save to local chromadb

In [7]:
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("aws_documentation_test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)
#load from local chromadb

In [8]:
from llama_index.prompts.prompts import SimpleInputPrompt

#System prompt will need major finetuning. We should aim to use as less tokens as possible to leave some for the QA. 
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided. Be patient, clear, and thorough in your explanations. You can ask for more information if needed. You will get tipped $1,000,000 for doing a good job. You will be penalized for doing a bad job."

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("What is alexa for business?")
display(Markdown(f"<b>{response}</b>"))

<b> Alexa for Business makes it easy for you to use Alexa in your organization, giving you the tools you need to manage Alexa devices, enroll your users, and assign skills at scale.</b>